In [1]:
from skimage import data, draw,color, data, restoration
from skimage import transform, util
from scipy import misc, fftpack, signal, ndimage
import numpy as np
from numpy.linalg import inv
from skimage import filters, color
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance
import matplotlib.animation as animation
from scipy.misc import imresize
from scipy import ndimage as nd
from random import randint
import scipy
scale = 4
rgb =  plt.imread('casa.jpg')
rgb = imresize(rgb, ( rgb.shape[0]/scale,rgb.shape[1]/scale,3),interp='bilinear')
img_LAB = color.rgb2lab(rgb.astype('float'))
img_gris = color.rgb2gray(rgb)
def normalitzar(imatge):
    return np.real((imatge - np.min(imatge)) * 256 / (np.max(imatge) - np.min(imatge)))
rgb=normalitzar(rgb)
img_gris=normalitzar(img_gris)

In [2]:
class get_mouse_click():
    
    #Mouse interaction interface for radial distortion removal.
    def __init__(self, img):
      height, width = img.shape[:2]
      self.figure = plt.imshow(img, extent=(0, width, height, 0))
      plt.gray()
      plt.title('select the object to remove')
      plt.xlabel('Select sets of  points with left mouse button,\n'
                 'click right button to close the polygon.')
      plt.connect('button_press_event', self.button_press)
      plt.connect('motion_notify_event', self.mouse_move)

      self.img = np.atleast_3d(img)
      self.points = []
      self.centre = np.array([(width - 1)/2., (height - 1)/2.])

      self.height = height
      self.width = width

      self.make_cursorline()
      self.figure.axes.set_autoscale_on(False)

      plt.show()
      plt.close()

    def make_cursorline(self):
        self.cursorline, = plt.plot([0],[0],'r:+', linewidth=2,markersize=15,markeredgecolor='b')

    def button_press(self,event):
        # Register mouse clicks.
        if (event.button == 1 and event.xdata and event.ydata):
            self.points.append((event.xdata,event.ydata))
            plt.gca().lines.append(self.cursorline)
            plt.gca()
            self.make_cursorline()
        if (event.button != 1):
            self.points.append((self.points[0][0],self.points[0][1]))
            plt.close()
            return self.points

    def mouse_move(self,event):
        # Handle cursor drawing.
        pts_last_set=len(self.points)
        pts = np.zeros((pts_last_set+1,2))
        if pts_last_set > 0:
            # Line follows up to 3 clicked points:
            pts[:pts_last_set] = self.points[-pts_last_set:]
            # The last point of the line follows the mouse cursor
        pts[pts_last_set:] = [event.xdata,event.ydata]
        self.cursorline.set_data(pts[:,0], pts[:,1])
        plt.draw()
def compute_mask(width,height,polygon):
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    #mask100 = 100*np.ones([mask.shape[0],mask.shape[1]])
    #mask=mask*101
    #mask = 1. - mask; # switch 0s and 1s
    return mask
def convine_mask(zona_delete,gris):
    #si tenim 1 a la macara tenim forat
    for i in xrange(len(gris)):
        for j in xrange(len(gris[0])):
            if zona_delete[i][j]==1:
                gris[i][j]=-100
    return gris
def reccorrer_img(finestra,finestra_validar,img_gris,t,r,vent):
    dif=19999999990.
    #valors=[]
    for i in xrange(len(img_gris)-vent):
        for j in xrange(len(img_gris[0])-vent):
            #valor = millors_pix(finestra,finestra_validar,img_gris,i,j)
            #si la puntuació es inferior al valor es que es prou semblant
            valor = millors_pix(finestra,finestra_validar,img_gris,i,j)
            
            if valor < dif:
                #print valor, dif, i, j
                valors = i,j
                dif = valor
    #print "i i j",i,j
    return valors
def millors_pix(finestra,finestra_validar,img_gris,t,r):
    #print finestra.shape
    #print finestra_validar.shape
    ssd=0.
    total_ones_mask = finestra_validar.sum()
    #aqui fariem el calcul de la formula explicada pel professor
    #1/numero de 1 de la mascara*(Ixy-Fxy)² ssd = float(np.sum((img_gris[:,:,0:3] - finestra[:,:,0:3])**2))
    for i in xrange(len(finestra_validar)):
        for j in xrange(len(finestra_validar[0])):
            ssd += (finestra_validar[i][j]*((img_gris[t+i,r+j] - finestra[i][j])**2))

    #print finestra_validar
    #print (1./total_ones_mask)*ssd
    return (1./total_ones_mask)*ssd

In [5]:
img_gris = color.rgb2gray(rgb)
img_gris=normalitzar(img_gris)
rdi = get_mouse_click(img_gris.astype('uint8'))
height,width=img_gris.shape
zona_delete = compute_mask(width,height,rdi.points)

vent=4
cont=0
img_gris_convine=img_gris
while zona_delete.sum()!=0:
    
    zona_delete_erosio=ndimage.binary_erosion(zona_delete)

    #busco el contorn mes extern de la part a substituir
    delete_ceva =zona_delete - zona_delete_erosio 
    #actualitzo la mascara amb el que ja haurem substituit
    
    finestra = np.zeros((vent,vent), dtype=np.int)
    img_gris_convine=convine_mask(zona_delete,img_gris_convine)
    for i in xrange(len(delete_ceva)):
        for j in xrange(len(delete_ceva[0])):
            if delete_ceva[i][j]==1:
                delete_ceva[i][j]=0
                finestra_validar = np.ones((vent,vent), dtype=np.int)
                finestra=img_gris_convine[i-vent/2:i+vent/2,j-vent/2:j+vent/2]               
                finestra_validar=abs(finestra-finestra_validar)
                #fig = plt.figure(123) 
                #plt.imshow(delete_ceva)
                #plt.title('delete_ceva')
                #plt.show()
                #print "finestra",finestra,"finestra validar", finestra_validar
                t,r = reccorrer_img(finestra,finestra_validar,img_gris_convine,i,j,vent)
                img_gris[i][j]=img_gris[t][r]
                misc.imsave('imatges/'+str(cont+1)+'.png',img_gris)
                cont= cont +1
                zona_delete=zona_delete_erosio
    

In [ ]:
img_gris_convine.shape
            

In [6]:
zona_delete.sum()

36

In [6]:
    fig = plt.figure(123) 
    plt.imshow(img_gris)
    plt.title('Contorn')
    plt.show()